In [7]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import os

In [3]:
LEARNING_RATE = 5e-5
BATCH_SIZE = 4096
N_EPOCHS = 5
dataDirectory = 'dataSet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device = ", device)

Device =  cuda


In [5]:
if not os.path.exists(dataDirectory):
    os.makedirs(dataDirectory)

downLoad = False
if not os.path.exists(os.path.join(dataDirectory, 'MNIST')):
    downLoad = True
trainData = datasets.MNIST(
    root = dataDirectory,
    train = True,
    transform = ToTensor(),
    download=downLoad
)
testData = datasets.MNIST(
    root = dataDirectory,
    train = False,
    transform = ToTensor(),
    download=downLoad
)

NameError: name 'ToTensor' is not defined

In [ ]:
loaders = {
    'trainLoader' : DataLoader()
}

In [18]:

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the training data
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8192, shuffle=True)

# Download and load the test data
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8192, shuffle=False)

# Print shapes of the datasets
images, labels = next(iter(trainloader))
print(f"Image batch shape: {images.size()}")
print(f"Label batch shape: {labels.size()}")

Image batch shape: torch.Size([8192, 1, 28, 28])
Label batch shape: torch.Size([8192])


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [25]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)  # Move to GPU

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(trainloader)}")

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)  # Move to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total} %")


Epoch 1 - Training loss: 1.988330826163292
Epoch 2 - Training loss: 1.0943696424365044
Epoch 3 - Training loss: 0.5855293646454811
Epoch 4 - Training loss: 0.42172203958034515
Epoch 5 - Training loss: 0.35730716586112976
Accuracy of the network on the 10000 test images: 90.57 %
